In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [41]:
dir = "/content/drive/MyDrive"

In [42]:
import numpy as np
masks = np.load(f"{dir}/masks.npy")
images = np.load(f"{dir}/images.npy")
masks = masks / 255.0

In [43]:
print(masks.shape)
print(images.shape)
print(np.unique(masks))

(3929, 256, 256)
(3929, 256, 256, 3)
[0. 1.]


In [44]:
from sklearn.model_selection import train_test_split

train_images, test_images, train_masks, test_masks = train_test_split(
    images, masks,
    test_size=0.01,
    random_state=42,
    shuffle=True
)
train_masks  = train_masks[..., np.newaxis]
test_masks = test_masks[..., np.newaxis]

In [45]:
print(test_masks.shape)
print(test_images.shape)

print(train_masks.shape)
print(train_images.shape)

(40, 256, 256, 1)
(40, 256, 256, 3)
(3889, 256, 256, 1)
(3889, 256, 256, 3)


In [46]:
import tensorflow as tf

def loss(true, pred):
  ce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  return ce(true, pred)

def metrics(y_true, y_pred):
    y_pred = tf.sigmoid(y_pred)
    y_pred = tf.round(y_pred)

    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection

    return intersection / (union + 1e-7)


In [47]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, ReLU, UpSampling2D, Concatenate, Input
from tensorflow.keras.models import Model

def conv(f, k, x):
  x = Conv2D(f, k, padding="same")(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  return x


size = 256

input = Input((size, size, 3))

x = conv(32, 3, input)
skip256=x
x = MaxPooling2D(2)(x)

x = conv(64, 3, x)
skip128=x
x = MaxPooling2D(2)(x)

x = conv(128, 3, x)
skip64=x
x = MaxPooling2D(2)(x)

x = conv(256, 3, x)
skip32=x
x = MaxPooling2D(2)(x)

x = conv(512, 3, x)
skip16=x
x = MaxPooling2D(2)(x)

x = conv(1024, 3, x)
skip8=x
x = MaxPooling2D(2)(x)

x = conv(2048, 3, x)

x = UpSampling2D(2)(x)
x = Concatenate()([x, skip8])
x = conv(1024, 3, x)

x = UpSampling2D(2)(x)
x = Concatenate()([x, skip16])
x = conv(512, 3, x)

x = UpSampling2D(2)(x)
x = Concatenate()([x, skip32])
x = conv(256, 3, x)

x = UpSampling2D(2)(x)
x = Concatenate()([x, skip64])
x = conv(128, 3, x)

x = UpSampling2D(2)(x)
x = Concatenate()([x, skip128])
x = conv(128, 3, x)

x = UpSampling2D(2)(x)
x = Concatenate()([x, skip256])
x = conv(128, 3, x)

output = Conv2D(1, 1)(x)

model = Model(input, output)
model.summary()

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    "best_keras.keras",
    monitor="val_metrics",
    save_best_only=True,
    mode="max",
    verbose=1
)

model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=loss,
              metrics=[metrics])

model.fit(train_images, train_masks,
          batch_size=8,
          epochs=100,
          validation_split=0.1,
          callbacks=[checkpoint])

model.save("keras.keras")



Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10      │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_140 (Conv2D) │ (None, 256, 256,  │        896 │ input_layer_10[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 256,  │        128 │ conv2d_140[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_60    │ (None, 128, 128,  │          0 │ re_lu[0][0]       │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_141 (Conv2D) │ (None, 128, 128,  │     18,496 │ max_pooling2d_60… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ conv2d_141[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_61    │ (None, 64, 64,    │          0 │ re_lu_1[0][0]     │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_142 (Conv2D) │ (None, 64, 64,    │     73,856 │ max_pooling2d_61… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        512 │ conv2d_142[0][0]  │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 64, 64,    │          0 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_62    │ (None, 32, 32,    │          0 │ re_lu_2[0][0]     │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_143 (Conv2D) │ (None, 32, 32,    │    295,168 │ max_pooling2d_62… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │      1,024 │ conv2d_143[0][0]  │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_63    │ (None, 16, 16,    │          0 │ re_lu_3[0][0]   

 Total params: 63,198,657 (241.08 MB)

 Trainable params: 63,186,177 (241.04 MB)

 Non-trainable params: 12,480 (48.75 KB)

Epoch 1/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0995 - metrics: 0.2221
Epoch 1: val_metrics improved from -inf to 0.26005, saving model to best_keras.keras
438/438 ━━━━━━━━━━━━━━━━━━━━ 51s 75ms/step - loss: 0.0994 - metrics: 0.2222 - val_loss: 0.0972 - val_metrics: 0.2600
Epoch 2/100
437/438 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0201 - metrics: 0.4343
Epoch 2: val_metrics did not improve from 0.26005
438/438 ━━━━━━━━━━━━━━━━━━━━ 17s 38ms/step - loss: 0.0201 - metrics: 0.4344 - val_loss: 0.0904 - val_metrics: 0.1692
Epoch 3/100
437/438 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0159 - metrics: 0.4909
Epoch 3: val_metrics improved from 0.26005 to 0.38334, saving model to best_keras.keras
438/438 ━━━━━━━━━━━━━━━━━━━━ 20s 45ms/step - loss: 0.0159 - metrics: 0.4909 - val_loss: 0.0188 - val_metrics: 0.3833
Epoch 4/100
437/438 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0144 - metrics: 0.5268
Epoch 4: val_metrics improved from 0.38334 to 0.56603, saving model to best_ke

In [52]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


# ======================
# 1. Load Best Model
# ======================
model = tf.keras.models.load_model("best_keras.keras", compile=False)
test_images = np.load("/content/drive/MyDrive/test_images.npy")
test_masks = np.load("/content/drive/MyDrive/test_masks.npy")

# ======================
# 2. Make Sure Masks Shape Is Correct
# ======================
if len(test_masks.shape) == 3:
    test_masks = test_masks[..., np.newaxis]

# ======================
# 3. Predict First 5 Images
# ======================
num = 20
preds = model.predict(test_images[:num])

# If model last layer has sigmoid
preds = (preds > 0.5).astype("float32")

# ======================
# 4. Show Results
# ======================
for i in range(num):
    plt.figure(figsize=(12,4))

    # Original Image
    plt.subplot(1,3,1)
    plt.imshow(test_images[i])
    plt.title("Image")
    plt.axis("off")

    # Ground Truth
    plt.subplot(1,3,2)
    plt.imshow(test_masks[i].squeeze(), cmap="gray")
    plt.title("Ground Truth")
    plt.axis("off")

    # Prediction
    plt.subplot(1,3,3)
    plt.imshow(preds[i].squeeze(), cmap="gray")
    plt.title("Prediction")
    plt.axis("off")

    plt.show()

Output hidden; open in https://colab.research.google.com to view.